# Sneak Peak into Sentiwordnet

In [1]:
from nltk.corpus import sentiwordnet as swn

In [2]:
swn.senti_synsets('dog')

[SentiSynset('dog.n.01'),
 SentiSynset('frump.n.01'),
 SentiSynset('dog.n.03'),
 SentiSynset('cad.n.01'),
 SentiSynset('frank.n.02'),
 SentiSynset('pawl.n.01'),
 SentiSynset('andiron.n.01'),
 SentiSynset('chase.v.01')]

In [3]:
# Each Synset has a polarity ~ Average them or use most common, as polarity of word

In [4]:
#SentiSynset('cad.n.01') -> dog represented as cad which is noun and have polarty.

In [5]:
swn.senti_synsets('dog')[3]

SentiSynset('cad.n.01')

In [6]:
swn.senti_synsets('dog')[3].pos_score()

0.0

In [7]:
swn.senti_synsets('dog')[3].neg_score()

1.0

# Implementing A Rule-based Approach using Sentiwordnet

In [8]:
#1. Taking most common meaning for a word to compute polarity

In [17]:
def superNaiveSentiment(review):
    reviewPolarity =0.0
    numExceptions= 0
    for word in review.lower().split():
        weight  = 0.0
        try:
            common_meaning = swn.senti_synsets(word)[0]
            if common_meaning.pos_score()>common_meaning.neg_score():
                weight= weight + common_meaning.pos_score()
            elif common_meaning.pos_score()<common_meaning.neg_score():
                weight= weight - common_meaning.neg_score()
        except :
            numExpections  = numExceptions+1
    #print "Word : "+ word +"weight : " + str(weight)
    reviewPolarity = reviewPolarity + weight
    return reviewPolarity

In [10]:
positiveReview = "C:\\Users\\praveen ji\\Desktop\\requiredDoc\\PyCode\\Dataset\\rt-polaritydata\\rt-polaritydata\\rt-polarity.pos"
with open(positiveReview,'r') as f:
    positiveReviews = f.readlines()

In [11]:
negativeReview = "C:\\Users\\praveen ji\\Desktop\\requiredDoc\\PyCode\\Dataset\\rt-polaritydata\\rt-polaritydata\\rt-polarity.neg"
with open(negativeReview,'r') as f:
    negativeReviews = f.readlines()

In [12]:
def getReviewSentiments(sentimentCalclator):
    negReviewResult = [sentimentCalclator(oneNegativeReview) for oneNegativeReview in negativeReviews ]
    posReviewResult = [sentimentCalclator(onePositiveReview) for onePositiveReview in positiveReviews ]
    return {'results-on-positive': posReviewResult , 'results-on-negative': negReviewResult}

In [13]:
def runDiagnostics(reviewResult):
    positiveReviewsResult = reviewResult['results-on-positive']
    negativeReviewsResult = reviewResult['results-on-negative']
    
    pctTruePositive = float(sum(x>0 for x in positiveReviewsResult))/len(positiveReviewsResult)
    pctTrueNegative = float(sum(x>0 for x in negativeReviewsResult))/len(negativeReviewsResult)
    
    totalAccurate = float(sum(x>0 for x in positiveReviewsResult))+ float(sum(x<0 for x in negativeReviewsResult))
    
    total = len(positiveReviewsResult) +len(negativeReviewsResult)
    
    print "Accuracy on positive reviews = " + "%.2f" % (pctTruePositive*100) + "%"
    print "Accuracy on negative reviews = " + "%.2f" % (pctTrueNegative*100) + "%"
    print "Accuracy Overall = " + "%.2f" % (totalAccurate*100/ total) + "%"
    

In [18]:
runDiagnostics(getReviewSentiments(superNaiveSentiment))

Accuracy on positive reviews = 0.32%
Accuracy on negative reviews = 0.36%
Accuracy Overall = 0.36%


In [19]:
#2. Taking avg of synsets for a word to compute polarity

In [ ]:
#1. stopword removal
#2. all meanings/lemmas are considered associated with word

In [21]:
from string import punctuation
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english')+ list(punctuation))

In [20]:
def NaiveSentiment(review):
    reviewPolarity =0.0
    numExceptions= 0
    for word in review.lower().split():
        numMeanings =0
        if word in stopwords:
            continue
        weight  = 0.0
        try:
            common_meaning = swn.senti_synsets(word)[0]
            if common_meaning.pos_score()>common_meaning.neg_score():
                weight= weight + (common_meaning.pos_score() - common_meaning.neg_score())
                numMeanings = numMeanings+1
            elif common_meaning.pos_score()<common_meaning.neg_score():
                weight= weight - (common_meaning.neg_score()- common_meaning.pos_score())
                numMeanings = numMeanings+1
        except :
            numExpections  = numExceptions+1
    #print "Word : "+ word +"weight : " + str(weight)
    if numMeanings>0:
        reviewPolarity = reviewPolarity + (weight/numMeanings)
    return reviewPolarity

In [23]:
runDiagnostics(getReviewSentiments(NaiveSentiment))

Accuracy on positive reviews = 0.30%
Accuracy on negative reviews = 0.36%
Accuracy Overall = 0.33%
